In [120]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pickle
import os
from PIL import ImageTk, Image, ImageDraw, ImageOps
import PIL
from tkinter import *

#This is the math part------------------------------------
if not os.path.isfile("HandWrittenDigitPredictor.p"):
    print("Reading Data...")
    mnist = np.genfromtxt('mnist_data.csv', delimiter=',')
    mnist = np.delete(mnist, 0, 0)
    print("Data Loaded")

    print("Processing Data")
    np.random.shuffle(mnist)
    train_data, test_data, train_label, test_label = train_test_split(mnist[:,:-1], mnist[:,-1], test_size=1/7.0, random_state=0)
    print("Data Processed")
    print("Loading Model...")
    logReg = LogisticRegression(solver = 'saga')
    print("Model Loaded\nStarting Training...")
    logReg.fit(train_data, train_label)
    print("Training Finished")
    with open("HandWrittenDigitPredictor.p", "wb") as f:
        pickle.dump(logReg, f)
    score = logReg.score(test_data, test_label)
    print(score)
else:
    with open("HandWrittenDigitPredictor.p", "rb") as f:
        logReg = pickle.load(f)
#---------------------------------------------------------

#This is the fancy GUI part-------------------------------

width = 400
height = width
WHITE = (255, 255, 255)
    
def paint(event):
    offset = 20
    x1, y1 = (event.x - offset), (event.y - offset)
    x2, y2 = (event.x + offset), (event.y + offset)
    cv.create_oval(x1, y1, x2, y2, fill="black")
    draw.ellipse( [x1, y1, x2, y2],fill="black")
    
def generateCompInput():
    small_image = ImageOps.invert(image1.resize((20,20)))
    
    pixels = np.array(small_image.getdata())
    pixels = np.amax(pixels, axis = 1)
    pixels.shape = (20,20)
    s = pixels.sum()
    x = pixels.sum(axis=0)
    y = pixels.sum(axis = 1)
    
    cmx, cmy = 0, 0
    for i in range(x.size):
        cmx += x[i]*i
    for i in range(y.size):
        cmy += y[i]*i
    cmx /= s
    cmy /= s

    centered_image = PIL.Image.new("L", (28,28))
    centered_image.paste(small_image, (14-int(cmx), 14-int(cmy)))
    centered_image.save("image.png")
    return centered_image

def predict():
    centered_image = generateCompInput()
    pixels = np.array(centered_image.getdata())
    pixels.shape = (1,784)
    print()
    photo2 = ImageTk.PhotoImage(centered_image.resize( (28*6,28*6)))
    mv_img.configure(image=photo2)
    mv_img.image = photo2
    p_ans.configure(text = str(int(logReg.predict(pixels))))
    
def clear():
    draw.rectangle((0,0,width,height), fill= WHITE)
    cv.delete("all")
    mv_img.configure(image=photo)
    mv_img.image = photo
    p_ans.configure(text = "_")

image1 = PIL.Image.new("RGB", (width, height), WHITE)
draw = ImageDraw.Draw(image1)

root = Tk()
c_frame = Frame(root)

c_lab = Label(c_frame, text = "Digit Predictor:", font=("Courier", 30))
c_lab.pack()
cv = Canvas(c_frame, width=width, height=height, bg='white')
cv.pack()
cv.pack(expand=NO, fill=BOTH, side="left")
cv.bind("<B1-Motion>", paint)

c_frame.pack(side="left")


predict_button = Button(text="Predict!", command =predict, font=("Courier", 22))
predict_button.pack()
    
clear_b = Button(text=" Clear! ", command = clear, font=("Courier", 22))
clear_b.pack()

mv_frame = Frame(root)
mv_lab = Label(mv_frame, text = "Function Input:", font=("Courier", 16))
mv_lab.pack()
photo = ImageTk.PhotoImage(PIL.Image.new("L", (28*6,28*6)))
mv_img = Label(mv_frame, image=photo)
mv_img.pack()
mv_frame.pack()

p_frame = Frame(root)
p_lab = Label(p_frame, text = "Prediction:", font=("Courier", 22))
p_lab.pack()
p_ans = Label(p_frame, text = "_", font=("Courier", 30))
p_ans.pack()
p_frame.pack()

root.mainloop()

1.0
